In [1]:
import astropy.time
from lsst.daf.butler import Butler, Timespan
import os
os.umask(222)

18

In [2]:
butler = Butler('/repo/embargo')
registry = butler.registry

In [3]:
scratch_butler = Butler('/home/r/rnevin/scratch', writeable=True)
scratch_registry = scratch_butler.registry

In [4]:
datasetType = 'deepCoadd'
collections = 'LATISS/runs/AUXTEL_DRP_IMAGING_2023-08A-07AB-05AB/w_2023_33/PREOPS-3613'
dataId = {'band':'i'}

In [5]:
now = astropy.time.Time.now().tai
#now_raw = "2023-09-16 01:35:30.002737" #01:35:48.002737
#now = astropy.time.Time(now_raw, scale="tai", format="iso")
embargo_hours = 80.0
embargo_period = astropy.time.TimeDelta(
        embargo_hours * 3600.0, format="sec"
    )
timespan_embargo = Timespan(now - embargo_period, None)
# timespan_embargo = Timespan(now - embargo_period, now)

In [6]:
now

<Time object: scale='tai' format='datetime' value=2023-10-06 16:42:30.160825>

In [7]:
timespan_embargo

Timespan(begin=astropy.time.Time('2023-10-03 08:42:30.160825', scale='tai', format='iso'), end=None)

In [8]:
timespan_embargo.begin

<Time object: scale='tai' format='iso' value=2023-10-03 08:42:30.160825>

In [9]:
datasetRefs = registry.queryDatasets(
            datasetType=datasetType,
            collections=collections,
            where="ingest_date <= timespan_embargo_begin",
            bind={"timespan_embargo_begin": timespan_embargo.begin},
        )

In [10]:
ids_to_move = [dt.id for dt in datasetRefs]
print(len(ids_to_move))
print(ids_to_move[0:10])

363
[UUID('e5c9aa36-3634-4aa7-b4fa-214f5a72ad4b'), UUID('d772e2f9-0741-4c00-b8d9-3d9bdd7da305'), UUID('c5fffb5e-b166-43b1-bad1-d5afcc916657'), UUID('72744dfb-eef5-47e6-9fed-ea3e255846d7'), UUID('5232cbaa-25a8-4ddb-b98c-d3983c0e184a'), UUID('d99bdf5b-f818-4c3e-a073-3705c787a8de'), UUID('f66da4ff-00e5-4b50-98d6-024a6fa3b121'), UUID('5e4dfb0d-aba9-4131-9f5d-e39e25d487e5'), UUID('0c38dcb5-9e76-4b9a-9f65-344d27fdb8aa'), UUID('cb263364-0034-4786-b9de-2b90c9c0d91f')]


In [ ]:
for i, dt in enumerate(registry.queryDatasets(datasetType=datasetType, collections=collections, 
                                                      where="ingest_date < T'2023-09-28 18:26:02.626054'",
                                                     )):
    print(dt)
    if i > 10:
        break

In [ ]:
for i, dt in enumerate(registry.queryDatasets(datasetType=datasetType, collections=collections, 
                                                      where="ingest_date < T'2023-09-28 18:26:02.626054' AND ingest_date > T'2023-08-18 02:45:53.173467'",
                                                     )):
    print(dt.id)
    if i > 10:
        break

In [ ]:
timespan_embargo

In [ ]:
for i, dt in enumerate(registry.queryDatasets(datasetType=datasetType, collections=collections, 
                                                      where="ingest_date < timespan_embargo_begin",
                                                     bind={"timespan_embargo_begin": timespan_embargo.begin},
                                             )):
    print(dt)
    if i > 10:
        break

In [ ]:
for i, dt in enumerate(registry.queryDatasets(datasetType=datasetType, collections=collections, 
                                                      where="ingest_date < timespan_embargo_end AND ingest_date > timespan_embargo_begin",
                                                     bind={"timespan_embargo_begin": timespan_embargo.begin, "timespan_embargo_end": timespan_embargo.end},
                                             )):
    print(dt)
    if i > 10:
        break

In [ ]:
now = astropy.time.Time.now().tai
embargo_hours = 1000.0
embargo_period = astropy.time.TimeDelta(
        embargo_hours * 3600.0, format="sec"
    )
timespan_embargo = Timespan(now - embargo_period, None)
# timespan_embargo = Timespan(now - embargo_period, now)

In [ ]:
for i, dt in enumerate(registry.queryDatasets(datasetType=datasetType, collections=collections, 
                                                      where="ingest_date < timespan_embargo_end AND ingest_date > timespan_embargo_begin",
                                                     bind={"timespan_embargo_begin": timespan_embargo.begin, "timespan_embargo_end": timespan_embargo.end},
                                             )):
    print(dt.id)
    if i > 10:
        break

In [ ]:
# for dt in registry.queryDimensionRecords("exposure",datasets=datasetType,collections=collections,
#             where="exposure.timespan OVERLAPS timespan_embargo",
#             bind={"timespan_embargo": timespan_embargo},
#         ):
#     print(dt.id)
#     if i > 10:
#         break

## check if exposure and vist dimension exist

In [ ]:
datasetType = 'goodSeeingCoadd'
collections = 'LATISS/runs/AUXTEL_DRP_IMAGING_2023-08ABC-07AB-05AB/w_2023_35/PREOPS-3726/20230912T151732Z'

In [ ]:
for dt in sorted(registry.queryDatasetTypes(datasetType)):
    print(dt)

In [ ]:
datasetType = 'raw'
collections = 'LATISS/raw/all/'

In [ ]:
for dt in sorted(registry.queryDatasetTypes(datasetType)):
    print(dt)

In [ ]:
# for c in sorted(registry.queryCollections(datasetType='calexp')):
#     print(c)

In [ ]:
datasetType = 'calexp'
collections = 'LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616'
for dt in sorted(registry.queryDatasetTypes(datasetType)):
    print(dt)

In [ ]:
for dt in sorted(registry.queryDatasetTypes(datasetType)):
    print(dt)

In [ ]:
# for dt in sorted(registry.queryDatasetTypes()):
#     print(dt)

In [ ]:
for dt in sorted(registry.queryDatasetTypes('calexp')):
    print(dt)

In [ ]:
for dt in sorted(registry.queryDatasetTypes('calexp.*')):
    print(dt)

In [ ]:
for i, dt in enumerate(registry.queryDimensionRecords('exposure',datasets=datasetType, collections=collections,
                                             )):
    print(dt)
    if i > 10:
        break

In [ ]:
for i, dt in enumerate(registry.queryDimensionRecords('visit',datasets=datasetType, collections=collections,
                                             )):
    print(dt)
    if i > 10:
        break

In [ ]:
for i, dt in enumerate(registry.queryDimensionRecords('visit',datasets=='calexp', collections=collections, 
                                                      where="visit.time < timespan_embargo_end AND ingest_date > timespan_embargo_begin",
                                                     bind={"timespan_embargo_begin": timespan_embargo.begin, "timespan_embargo_end": timespan_embargo.end},
                                             )):
    print(dt.id)
    if i > 10:
        break